In [1]:
import os
from pathlib import Path

# 📁 Ruta raíz donde están las ciudades
# 👉 Asegúrate de que esta carpeta contiene las subcarpetas de las escenas (no te metas dentro de una)
ruta_raiz = Path("/home/rohi/Downloads/maestria/10-heat_island")

# 🔍 Buscar recursivamente carpetas que tengan el archivo de temperatura modelo
carpetas_validas = []
for root, dirs, files in os.walk(ruta_raiz):
    files_lower = [f.lower() for f in files]
    # Se aceptan variantes de mayúsculas/minúsculas y .tif / .TIF
    if any(f.startswith("temperature_extratrees") and f.endswith(".tif") for f in files_lower):
        carpetas_validas.append(Path(root))

# 📊 Reporte
print(f"Se encontraron {len(carpetas_validas)} carpetas válidas:\n")
for c in carpetas_validas:
    print("-", c)


Se encontraron 30 carpetas válidas:

- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20240120_20240129_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20190122_20200830_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20170116_20200905_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20230117_20230131_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20180204_20200902_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20160114_20200907_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20150401_20200909_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_010053_20220222_20220302_02_T1
- /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_2021021

In [2]:
import os
from pathlib import Path
import rasterio
import numpy as np

def _resolve_file(carpeta: Path, filename: str) -> Path:
    """Busca un archivo por nombre exacto ignorando mayúsculas/minúsculas (.tif/.TIF)."""
    target = filename.lower()
    for f in os.listdir(carpeta):
        if f.lower() == target:
            return carpeta / f
    raise FileNotFoundError(f"No se encontró {filename} en {carpeta}")

def _read1(path: Path):
    """Lee una banda (1) y retorna (array, profile)."""
    with rasterio.open(path) as src:
        arr = src.read(1)
        profile = src.profile
    return arr, profile

def cargar_rasters_y_mascaras(
    carpeta: Path,
    umbral_densidad_urbano: float = 0.6,
    umbral_densidad_veg: float = 0.6,
    umbral_agua_nube: float = 0.5
):
    """Lee los rásteres base y genera las máscaras de urbano, vegetación y exclusión, validando geometría/CRS."""
    print(f"\n📂 Procesando carpeta: {carpeta.name}")

    # --- Resolver nombres tolerantes a mayúsculas/minúsculas ---
    temp_model_path = _resolve_file(carpeta, "Temperature_ExtraTrees.TIF")
    lst_path        = _resolve_file(carpeta, "LST.TIF")
    sea_path        = _resolve_file(carpeta, "c0_sea_water.TIF")
    fresh_path      = _resolve_file(carpeta, "c1_fresh_water.TIF")
    builds_path     = _resolve_file(carpeta, "c2_builds.TIF")
    clouds_path     = _resolve_file(carpeta, "c3_clouds.TIF")
    bare_path       = _resolve_file(carpeta, "c4_bare_ground.TIF")
    veg_path        = _resolve_file(carpeta, "c5_vegetation.TIF")

    # --- Leer insumos (cerrando handlers) ---
    temp_model, profile = _read1(temp_model_path)
    lst, _       = _read1(lst_path)
    sea, _       = _read1(sea_path)
    fresh, _     = _read1(fresh_path)
    builds, _    = _read1(builds_path)
    clouds, _    = _read1(clouds_path)
    bare, _      = _read1(bare_path)
    vegetation, _= _read1(veg_path)

    # --- Validaciones geométricas básicas ---
    H, W = temp_model.shape
    def _chk(name, arr):
        if arr.shape != (H, W):
            raise ValueError(f"{name} tiene shape {arr.shape} distinto a {temp_model.shape} en {carpeta}")
    _chk("LST", lst); _chk("c0_sea_water", sea); _chk("c1_fresh_water", fresh)
    _chk("c2_builds", builds); _chk("c3_clouds", clouds); _chk("c4_bare_ground", bare); _chk("c5_vegetation", vegetation)

    # --- Construcción de máscaras ---
    urbano = (builds >= umbral_densidad_urbano).astype(np.uint8)
    vegetacion_densa = (vegetation >= umbral_densidad_veg).astype(np.uint8)
    agua = ((sea > umbral_agua_nube) | (fresh > umbral_agua_nube)).astype(np.uint8)
    nubes = (clouds > umbral_agua_nube).astype(np.uint8)
    mascara_exclusion = ((agua == 1) | (nubes == 1)).astype(np.uint8)

    # --- QC rápido ---
    tot_pix = H * W
    u_pix = int(urbano.sum()); v_pix = int(vegetacion_densa.sum()); exc_pix = int(mascara_exclusion.sum())
    print("Máscaras creadas correctamente:")
    print(f"  - Urbano: {u_pix} px ({100*u_pix/tot_pix:.2f}%)  [umbral {umbral_densidad_urbano}]")
    print(f"  - Vegetación densa: {v_pix} px ({100*v_pix/tot_pix:.2f}%)  [umbral {umbral_densidad_veg}]")
    print(f"  - Exclusión (agua/nubes): {exc_pix} px ({100*exc_pix/tot_pix:.2f}%)  [>{umbral_agua_nube}]")

    return {
        "temp_model": temp_model,
        "lst": lst,
        "urbano": urbano,
        "vegetacion": vegetacion_densa,
        "mascara_exclusion": mascara_exclusion,
        "profile": profile
    }


In [3]:
# Prueba con una carpeta
datos = cargar_rasters_y_mascaras(carpetas_validas[1])
assert set(["temp_model","lst","urbano","vegetacion","mascara_exclusion","profile"]) <= set(datos.keys())



📂 Procesando carpeta: LC08_L2SP_009053_20240120_20240129_02_T1
Máscaras creadas correctamente:
  - Urbano: 94582 px (12.71%)  [umbral 0.6]
  - Vegetación densa: 225379 px (30.28%)  [umbral 0.6]
  - Exclusión (agua/nubes): 264252 px (35.51%)  [>0.5]


In [4]:
import numpy as np
import rasterio
from pathlib import Path

def calcular_deltaT(datos, carpeta, usar_outputs=True, percentil_base=25):
    """
    Calcula la línea base térmica (percentil en vegetación densa sin exclusiones),
    genera DeltaT y lo guarda (opcionalmente) en /outputs.
    Retorna: (deltaT, T_base)
    """
    temp       = datos["temp_model"]           # Temperature_ExtraTrees.TIF
    vegetacion = datos["vegetacion"]           # c5_vegetation >= umbral -> 1
    exc        = datos["mascara_exclusion"]    # agua | nubes
    profile    = datos["profile"]

    # Carpeta de salida
    out_dir = Path(carpeta) if usar_outputs else Path(carpeta)
    out_dir.mkdir(exist_ok=True)

    # Máscara base: vegetación densa, sin exclusiones y valores finitos
    m_base = (vegetacion == 1) & (exc == 0) & np.isfinite(temp)
    if not np.any(m_base):
        raise ValueError("No hay píxeles válidos para calcular la línea base (vegetación densa sin nubes/agua).")

    # Línea base térmica
    T_base = np.nanpercentile(temp[m_base], percentil_base)
    print(f"🌡️  Línea base térmica (P{percentil_base}): {T_base:.2f} °C")

    # DeltaT
    deltaT = temp - T_base

    # Guardar DeltaT
    out_path = out_dir / "DeltaT.TIF"
    profile_f32 = profile.copy()
    profile_f32.update(dtype=rasterio.float32, nodata=np.nan, count=1, compress="lzw")
    with rasterio.open(out_path, "w", **profile_f32) as dst:
        dst.write(deltaT.astype(np.float32), 1)

    print(f"✅ DeltaT guardado en: {out_path}")
    return deltaT, T_base


In [5]:
deltaT, T_base = calcular_deltaT(datos, carpetas_validas[0])


🌡️  Línea base térmica (P25): 31.27 °C
✅ DeltaT guardado en: /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/DeltaT.TIF


In [6]:
def detectar_icu(datos, deltaT, carpeta, usar_outputs=True):
    import numpy as np
    import rasterio
    import json
    from pathlib import Path

    urbano = datos["urbano"]
    mascara_exclusion = datos["mascara_exclusion"]
    profile = datos["profile"]

    # Carpeta de salida
    output_dir = Path(carpeta)  if usar_outputs else Path(carpeta)
    output_dir.mkdir(exist_ok=True)

    # Área válida: urbano sin exclusiones y con ΔT finito
    mascara_valida = (urbano == 1) & (mascara_exclusion == 0)
    mascara_valid_values = mascara_valida & np.isfinite(deltaT)

    total_urbano = int(np.sum(mascara_valida))
    if total_urbano == 0:
        # Nada de urbano válido: escribir salidas vacías y salir
        icu_intensity = np.zeros_like(deltaT, dtype=np.uint8)
        icu_mask = np.zeros_like(deltaT, dtype=np.uint8)
        profile_u8 = profile.copy()
        profile_u8.update(dtype=rasterio.uint8, nodata=0, count=1, compress="lzw")
        with rasterio.open(output_dir / "ICU_mask.TIF", "w", **profile_u8) as dst:
            dst.write(icu_mask, 1)
        with rasterio.open(output_dir / "ICU_intensity.TIF", "w", **profile_u8) as dst:
            dst.write(icu_intensity, 1)
        print("ℹ️ No hay píxeles urbanos válidos. Archivos ICU vacíos escritos.")
        # metadata mínima
        with open(output_dir / "ICU_metadata.json", "w") as f:
            json.dump({"nota":"sin_urbano_valido","porcentaje_icu":0.0}, f, indent=2)
        return {
            "icu_mask": icu_mask, "icu_intensity": icu_intensity,
            "P75": np.nan, "P90": np.nan, "P97": np.nan,
            "porcentaje_icu": 0.0, "out_dir": str(output_dir)
        }

    # Percentiles sobre urbano válido (ignorando NaN)
    valores_validos = deltaT[mascara_valid_values]
    if valores_validos.size < 10:
        # Con muy pocos píxeles, fija umbral único alto para evitar falsos positivos
        P75 = P90 = P97 = float(np.nanmax(valores_validos)) if valores_validos.size > 0 else np.nan
    else:
        P75, P90, P97 = np.nanpercentile(valores_validos, [75, 90, 97])

    print("📊 Percentiles ΔT (urbano):")
    print(f"   P75 = {P75:.2f} °C | P90 = {P90:.2f} °C | P97 = {P97:.2f} °C")

    # Clasificación por intensidad SOLO en área válida
    icu_intensity = np.zeros_like(deltaT, dtype=np.uint8)
    if np.isfinite(P75) and np.isfinite(P90) and np.isfinite(P97):
        m = mascara_valid_values
        icu_intensity[(deltaT >= P75) & (deltaT < P90) & m] = 1  # Moderada
        icu_intensity[(deltaT >= P90) & (deltaT < P97) & m] = 2  # Alta
        icu_intensity[(deltaT >= P97) & m] = 3                   # Extrema

    # Mapa binario
    icu_mask = (icu_intensity > 0).astype(np.uint8)

    # Guardar
    profile_u8 = profile.copy()
    profile_u8.update(dtype=rasterio.uint8, nodata=0, count=1, compress="lzw")

    out_mask = output_dir / "ICU_mask.TIF"
    out_intensity = output_dir / "ICU_intensity.TIF"

    with rasterio.open(out_mask, "w", **profile_u8) as dst:
        dst.write(icu_mask, 1)
    with rasterio.open(out_intensity, "w", **profile_u8) as dst:
        dst.write(icu_intensity, 1)

    # Métricas
    total_icu = int(np.sum(icu_mask))
    porc_icu = (total_icu / total_urbano) * 100 if total_urbano > 0 else 0.0

    print(f"🏙️ Urbano válido: {total_urbano} px | 🔥 ICU: {total_icu} px ({porc_icu:.2f}%)")
    print(f"✅ Guardado en: {out_mask}\n✅ Guardado en: {out_intensity}")

    # Metadata JSON (trazabilidad)
    with open(output_dir / "ICU_metadata.json", "w") as f:
        json.dump({
            "P75": None if not np.isfinite(P75) else float(P75),
            "P90": None if not np.isfinite(P90) else float(P90),
            "P97": None if not np.isfinite(P97) else float(P97),
            "porcentaje_icu": float(porc_icu),
            "total_urbano_px": total_urbano,
            "total_icu_px": total_icu
        }, f, indent=2)

    return {
        "icu_mask": icu_mask,
        "icu_intensity": icu_intensity,
        "P75": P75, "P90": P90, "P97": P97,
        "porcentaje_icu": porc_icu,
        "out_dir": str(output_dir)
    }

In [7]:
# Escena de prueba punta-a-punta
carpeta_test = carpetas_validas[0]
d = cargar_rasters_y_mascaras(carpeta_test)
deltaT, T_base = calcular_deltaT(d, carpeta_test, usar_outputs=True, percentil_base=25)
res = detectar_icu(d, deltaT, carpeta_test, usar_outputs=True)
print("T_base:", round(T_base,2), "| %ICU:", round(res["porcentaje_icu"],2))



📂 Procesando carpeta: LC08_L2SP_009053_20200329_20200822_02_T1
Máscaras creadas correctamente:
  - Urbano: 134728 px (18.10%)  [umbral 0.6]
  - Vegetación densa: 32411 px (4.35%)  [umbral 0.6]
  - Exclusión (agua/nubes): 255798 px (34.37%)  [>0.5]
🌡️  Línea base térmica (P25): 32.09 °C
✅ DeltaT guardado en: /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/DeltaT.TIF
📊 Percentiles ΔT (urbano):
   P75 = -0.23 °C | P90 = -0.06 °C | P97 = 0.12 °C
🏙️ Urbano válido: 134728 px | 🔥 ICU: 33828 px (25.11%)
✅ Guardado en: /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/ICU_mask.TIF
✅ Guardado en: /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/ICU_intensity.TIF
T_base: 32.09 | %ICU: 25.11


In [8]:
import pandas as pd
import re
from pathlib import Path

# Fallback para tqdm si no está instalado
try:
    from tqdm import tqdm
except ModuleNotFoundError:
    def tqdm(x, **k): return x

def extraer_ciudad_y_fecha(path_str: str):
    p = Path(path_str)
    ciudad = p.parent.name                  # p.ej. cartagena / barranquilla / santa_marta
    m = re.search(r'_(\d{8})_', p.name)     # captura YYYYMMDD en nombre de escena
    fecha = m.group(1) if m else None
    return ciudad, fecha

def procesar_carpeta(carpeta: Path):
    """Ejecuta todos los pasos para una carpeta y devuelve un registro para el resumen."""
    try:
        # Paso 2
        datos = cargar_rasters_y_mascaras(carpeta)

        # Paso 3
        deltaT, T_base = calcular_deltaT(datos, carpeta, usar_outputs=True, percentil_base=25)

        # Paso 4
        resultados = detectar_icu(datos, deltaT, carpeta, usar_outputs=True)

        ciudad, fecha = extraer_ciudad_y_fecha(str(carpeta))
        return {
            "ciudad": ciudad,
            "fecha": fecha,
            "carpeta": str(carpeta),
            "T_base_P25": round(T_base, 3),
            "P75": round(resultados["P75"], 3) if resultados["P75"]==resultados["P75"] else None,
            "P90": round(resultados["P90"], 3) if resultados["P90"]==resultados["P90"] else None,
            "P97": round(resultados["P97"], 3) if resultados["P97"]==resultados["P97"] else None,
            "porc_icu": round(resultados["porcentaje_icu"], 3),
            "output_dir": resultados["out_dir"]
        }
    except Exception as e:
        print(f"⚠️  Error en {carpeta.name}: {e}")
        return None

# 🔁 Procesar todas las carpetas válidas
registros = []
for carpeta in tqdm(carpetas_validas, desc="Procesando escenas"):
    res = procesar_carpeta(carpeta)
    if res:
        registros.append(res)

# 📊 Crear resumen global ordenado
df = pd.DataFrame(registros)
cols = ["ciudad","fecha","carpeta","T_base_P25","P75","P90","P97","porc_icu","output_dir"]
df = df[cols] if not df.empty else df

csv_path = ruta_raiz / "resumen_ICU.csv"
df.to_csv(csv_path, index=False)

print(f"\n✅ Procesamiento completado.")
print(f"📄 Resumen global guardado en: {csv_path}")
display(df)



📂 Procesando carpeta: LC08_L2SP_009053_20200329_20200822_02_T1
Máscaras creadas correctamente:
  - Urbano: 134728 px (18.10%)  [umbral 0.6]
  - Vegetación densa: 32411 px (4.35%)  [umbral 0.6]
  - Exclusión (agua/nubes): 255798 px (34.37%)  [>0.5]
🌡️  Línea base térmica (P25): 32.09 °C
✅ DeltaT guardado en: /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/DeltaT.TIF
📊 Percentiles ΔT (urbano):
   P75 = -0.23 °C | P90 = -0.06 °C | P97 = 0.12 °C
🏙️ Urbano válido: 134728 px | 🔥 ICU: 33828 px (25.11%)
✅ Guardado en: /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/ICU_mask.TIF
✅ Guardado en: /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/ICU_intensity.TIF

📂 Procesando carpeta: LC08_L2SP_009053_20240120_20240129_02_T1
Máscaras creadas correctamente:
  - Urbano: 94582 px (12.71%)  [umbral 0.6]
  - Vegetación densa: 225379 px (30.28%)  [umbral 0.6]
  

,ciudad,fecha,carpeta,T_base_P25,P75,P90,P97,porc_icu,output_dir
0,cartagena,20200329,/home/rohi/Downloads/maestria/10-heat_island/c...,32.088001,-0.230,-0.058,0.121,25.108,/home/rohi/Downloads/maestria/10-heat_island/c...
1,cartagena,20240120,/home/rohi/Downloads/maestria/10-heat_island/c...,31.266001,0.073,0.177,0.325,25.001,/home/rohi/Downloads/maestria/10-heat_island/c...
2,cartagena,20190122,/home/rohi/Downloads/maestria/10-heat_island/c...,30.408001,0.400,0.427,0.468,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
3,cartagena,20170116,/home/rohi/Downloads/maestria/10-heat_island/c...,29.257000,0.535,0.621,0.682,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
4,cartagena,20230117,/home/rohi/Downloads/maestria/10-heat_island/c...,29.104000,0.355,0.545,0.707,25.001,/home/rohi/Downloads/maestria/10-heat_island/c...
5,cartagena,20180204,/home/rohi/Downloads/maestria/10-heat_island/c...,30.504000,-0.253,0.084,0.574,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
6,cartagena,20160114,/home/rohi/Downloads/maestria/10-heat_island/c...,29.424999,0.953,1.113,1.202,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
7,cartagena,20150401,/home/rohi/Downloads/maestria/10-heat_island/c...,30.415001,-0.040,0.180,0.456,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
8,cartagena,20220222,/home/rohi/Downloads/maestria/10-heat_island/c...,30.882000,0.002,0.147,0.454,25.001,/home/rohi/Downloads/maestria/10-heat_island/c...
9,cartagena,20210212,/home/rohi/Downloads/maestria/10-heat_island/c...,31.002001,-0.117,0.257,0.714,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...


In [28]:
import re

def extraer_ciudad_y_fecha(path_str: str):
    # ciudad = carpeta madre inmediata (ej. .../cartagena/LC08_.../ )
    p = Path(path_str)
    ciudad = p.parent.name  # cartagena / barranquilla / santa_marta / etc.

    # fecha = primera secuencia de 8 dígitos en el nombre de la escena
    m = re.search(r'_(\d{8})_', p.name)
    fecha = m.group(1) if m else None  # 'YYYYMMDD'
    return ciudad, fecha

# aplicar al dataframe ya creado (df)
df[["ciudad","fecha"]] = df["carpeta"].apply(lambda s: pd.Series(extraer_ciudad_y_fecha(s)))

# reordenar columnas
cols = ["ciudad","fecha","carpeta","T_base_P25","P75","P90","P97","porc_icu","output_dir"]
df = df[cols]

# guardar de nuevo
csv_path = ruta_raiz / "resumen_ICU.csv"
df.to_csv(csv_path, index=False)
display(df)


,ciudad,fecha,carpeta,T_base_P25,P75,P90,P97,porc_icu,output_dir
0,cartagena,20200329,/home/rohi/Downloads/maestria/10-heat_island/c...,32.088001,-0.230,-0.058,0.121,25.108,/home/rohi/Downloads/maestria/10-heat_island/c...
1,cartagena,20240120,/home/rohi/Downloads/maestria/10-heat_island/c...,31.266001,0.073,0.177,0.325,25.001,/home/rohi/Downloads/maestria/10-heat_island/c...
2,cartagena,20190122,/home/rohi/Downloads/maestria/10-heat_island/c...,30.408001,0.400,0.427,0.468,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
3,cartagena,20170116,/home/rohi/Downloads/maestria/10-heat_island/c...,29.257000,0.535,0.621,0.682,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
4,cartagena,20230117,/home/rohi/Downloads/maestria/10-heat_island/c...,29.104000,0.355,0.545,0.707,25.001,/home/rohi/Downloads/maestria/10-heat_island/c...
5,cartagena,20180204,/home/rohi/Downloads/maestria/10-heat_island/c...,30.504000,-0.253,0.084,0.574,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
6,cartagena,20160114,/home/rohi/Downloads/maestria/10-heat_island/c...,29.424999,0.953,1.113,1.202,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
7,cartagena,20150401,/home/rohi/Downloads/maestria/10-heat_island/c...,30.415001,-0.040,0.180,0.456,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
8,cartagena,20220222,/home/rohi/Downloads/maestria/10-heat_island/c...,30.882000,0.002,0.147,0.454,25.001,/home/rohi/Downloads/maestria/10-heat_island/c...
9,cartagena,20210212,/home/rohi/Downloads/maestria/10-heat_island/c...,31.002001,-0.117,0.257,0.714,25.000,/home/rohi/Downloads/maestria/10-heat_island/c...
